In [1]:
!pip install ortools==9.7.2996 pandas==2.1.0

In [2]:
from ortools.sat.python import cp_model

In [55]:
def criar_estacionamentos(total_de_estacionamentos, total_de_avioes, modelo):
  return [modelo.NewIntVar(0, total_de_avioes, f'estacionamento_{i}') for i in range(total_de_estacionamentos)]


In [106]:
def resolve(solucionador, modelo, estacionamentos, avioes):
  status = solucionador.Solve(modelo)
  print(solucionador.StatusName(status))
  if status == cp_model.INFEASIBLE:
    print("Sem solucao")
    return
  for estacionamento in estacionamentos:
    valor = solucionador.Value(estacionamento)
    if valor == 0:
      print(f"{estacionamento} sem aviao")
    else:
      aviao = avioes[valor - 1]
      print(f"{estacionamento} tem aviao {valor} grande={aviao.grande}")


In [107]:
def avioes_distintos(estacionamentos, modelo):
  modelo.AddAllDifferent(estacionamentos)

In [108]:
def todo_aviao_tem_que_estacionar(total_de_avioes, estacionamentos, modelo):
  variaveis = {}
  for i in range(1, total_de_avioes + 1):
    for j, estacionamento in enumerate(estacionamentos):
      aviao_i_em_j = modelo.NewBoolVar(f'aviao_{i}_em{j}')
      modelo.Add(estacionamento == i).OnlyEnforceIf(aviao_i_em_j)
      modelo.Add(estacionamento != i).OnlyEnforceIf(aviao_i_em_j.Not())
      variaveis[(i, j)] = aviao_i_em_j

  for i in range(1, total_de_avioes + 1):
    modelo.AddExactlyOne([variaveis[(i,j)] for j in range(len(estacionamentos))])


In [109]:
class Estacionamento:
  def __init__(self, grande):
    self.grande = grande

In [110]:
class Aviao:
  def __init__(self, k, grande):
    self.k = k
    self.grande = grande

In [111]:
def remover_do_estacionamento(modelo, variavel_estacionamento, avioes_grandes):
  for aviao in avioes_grandes:
    modelo.Add(variavel_estacionamento != aviao.k)

In [112]:
def limita_aviao_grande_para_estacionamento_grande(modelo, estacionamentos, avioes):
  avioes_grandes = [aviao for aviao in avioes if aviao.grande]
  for estacionamento in estacionamentos:
    if not estacionamento.grande:
      # nao aceita!!
      # REMOVER AVIOES GRANDES!
      remover_do_estacionamento(modelo, estacionamento.variavel, avioes_grandes)

In [113]:
def criar_estacionamentos(estacionamentos, total_de_avioes, modelo):
  variaveis = []
  for i, estacionamento in enumerate(estacionamentos):
    variavel = modelo.NewIntVar(0, total_de_avioes, f'estacionamento_{i}')
    estacionamento.variavel = variavel
    variaveis.append(variavel)
  return variaveis


In [115]:
avioes = [Aviao(1, True),
          Aviao(2, False)]
estacionamentos = [Estacionamento(False),
                   Estacionamento(False),
                   Estacionamento(True)]

modelo = cp_model.CpModel()
total_de_avioes = len(avioes)
variaveis_estacionamentos = criar_estacionamentos(estacionamentos, total_de_avioes, modelo)

avioes_distintos(variaveis_estacionamentos, modelo)
todo_aviao_tem_que_estacionar(total_de_avioes, variaveis_estacionamentos, modelo)
limita_aviao_grande_para_estacionamento_grande(modelo, estacionamentos, avioes)

solucionador = cp_model.CpSolver()
resolve(solucionador, modelo, variaveis_estacionamentos, avioes)

OPTIMAL
estacionamento_0 sem aviao
estacionamento_1 tem aviao 2 grande=False
estacionamento_2 tem aviao 1 grande=True


In [116]:
avioes = [Aviao(1, True),
          Aviao(2, False)]
estacionamentos = [Estacionamento(True),
                   Estacionamento(True),
                   Estacionamento(False)]

modelo = cp_model.CpModel()
total_de_avioes = len(avioes)
variaveis_estacionamentos = criar_estacionamentos(estacionamentos, total_de_avioes, modelo)

avioes_distintos(variaveis_estacionamentos, modelo)
todo_aviao_tem_que_estacionar(total_de_avioes, variaveis_estacionamentos, modelo)
limita_aviao_grande_para_estacionamento_grande(modelo, estacionamentos, avioes)

solucionador = cp_model.CpSolver()
resolve(solucionador, modelo, variaveis_estacionamentos, avioes)

OPTIMAL
estacionamento_0 tem aviao 1 grande=True
estacionamento_1 sem aviao
estacionamento_2 tem aviao 2 grande=False


In [117]:
avioes = [Aviao(1, True),
          Aviao(2, True)]
estacionamentos = [Estacionamento(True),
                   Estacionamento(True),
                   Estacionamento(False)]

modelo = cp_model.CpModel()
total_de_avioes = len(avioes)
variaveis_estacionamentos = criar_estacionamentos(estacionamentos, total_de_avioes, modelo)

avioes_distintos(variaveis_estacionamentos, modelo)
todo_aviao_tem_que_estacionar(total_de_avioes, variaveis_estacionamentos, modelo)
limita_aviao_grande_para_estacionamento_grande(modelo, estacionamentos, avioes)

solucionador = cp_model.CpSolver()
resolve(solucionador, modelo, variaveis_estacionamentos, avioes)

OPTIMAL
estacionamento_0 tem aviao 2 grande=True
estacionamento_1 tem aviao 1 grande=True
estacionamento_2 sem aviao


In [119]:
avioes = [Aviao(1, True),
          Aviao(2, True)]
estacionamentos = [Estacionamento(True),
                   Estacionamento(False),
                   Estacionamento(False)]

modelo = cp_model.CpModel()
total_de_avioes = len(avioes)
variaveis_estacionamentos = criar_estacionamentos(estacionamentos, total_de_avioes, modelo)

avioes_distintos(variaveis_estacionamentos, modelo)
todo_aviao_tem_que_estacionar(total_de_avioes, variaveis_estacionamentos, modelo)
limita_aviao_grande_para_estacionamento_grande(modelo, estacionamentos, avioes)

solucionador = cp_model.CpSolver()
resolve(solucionador, modelo, variaveis_estacionamentos, avioes)

INFEASIBLE
Sem solucao
